In [1]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
df = pd.read_csv('processed_training_data2.csv')


C:\Users\ferha\AppData\Local\Temp\ipykernel_12076\428823519.py:6: DtypeWarning: Columns (12,27,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_training_data2.csv')


In [2]:

# Get the list of column names
column_names = df.columns.tolist()

for i, column in enumerate(column_names):
    print(f"{i+1}. {column}")

column_names.remove('id')
column_names.remove('Degerlendirme Puani')


1. Basvuru Yili
2. Degerlendirme Puani
3. Cinsiyet
4. Dogum Yeri
5. Ikametgah Sehri
6. Universite Adi
7. Universite Turu
8. Burslu ise Burs Yuzdesi
9. Burs Aliyor mu?
10. Bölüm
11. Universite Kacinci Sinif
12. Universite Not Ortalamasi
13. Daha Once Baska Bir Universiteden Mezun Olmus
14. Lise Turu
15. Lise Mezuniyet Notu
16. Baska Bir Kurumdan Burs Aliyor mu?
17. Burs Aldigi Baska Kurum
18. Anne Egitim Durumu
19. Anne Calisma Durumu
20. Anne Sektor
21. Baba Egitim Durumu
22. Baba Calisma Durumu
23. Baba Sektor
24. Kardes Sayisi
25. Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?
26. Profesyonel Bir Spor Daliyla Mesgul musunuz?
27. Aktif olarak bir STK üyesi misiniz?
28. Stk Projesine Katildiniz Mi?
29. Girisimcilikle Ilgili Deneyiminiz Var Mi?
30. Ingilizce Biliyor musunuz?
31. Ingilizce Seviyeniz?
32. Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite
33. id
34. Yas
35. lise giris ort


In [3]:
df.drop(columns=['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'], inplace=True)

In [5]:
categorical_features = ['Basvuru Yili', 'Cinsiyet', 'Dogum Yeri', 'Ikametgah Sehri',
                        'Universite Adi', 'Universite Turu', 'Burs Aliyor mu?','Bölüm','Daha Once Baska Bir Universiteden Mezun Olmus',
                        'Lise Turu','Baska Bir Kurumdan Burs Aliyor mu?','Anne Calisma Durumu','Anne Sektor','Baba Calisma Durumu',
                        'Baba Sektor','Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?','Profesyonel Bir Spor Daliyla Mesgul musunuz?','Aktif olarak bir STK üyesi misiniz?',
                        'Stk Projesine Katildiniz Mi?','Girisimcilikle Ilgili Deneyiminiz Var Mi?',
                        'Ingilizce Biliyor musunuz?', 'Burs Aldigi Baska Kurum']

numerical_features = [column for column in df.columns if column not in categorical_features]
print(numerical_features)

# Fill missing values in categorical features
df[categorical_features] = df[categorical_features].fillna("0").astype(str)

# Fill missing values in numerical features
df[numerical_features] = df[numerical_features].fillna(0)


X = df.drop(['Degerlendirme Puani', 'id'], axis=1)  # Drop target and id columns
y = df['Degerlendirme Puani']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.fillna(y_train.mean())
y_val = y_val.fillna(y_val.mean())


['Degerlendirme Puani', 'Burslu ise Burs Yuzdesi', 'Universite Kacinci Sinif', 'Universite Not Ortalamasi', 'Lise Mezuniyet Notu', 'Anne Egitim Durumu', 'Baba Egitim Durumu', 'Kardes Sayisi', 'Ingilizce Seviyeniz?', 'id', 'Yas', 'lise giris ort']


In [6]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=7,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)



# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_x.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.6795978	test: 16.7242419	best: 16.7242419 (0)	total: 95.7ms	remaining: 4m 46s
100:	learn: 5.6869003	test: 5.7495598	best: 5.7495598 (100)	total: 5.27s	remaining: 2m 31s
200:	learn: 5.5378958	test: 5.7027816	best: 5.7027816 (200)	total: 10.3s	remaining: 2m 23s
300:	learn: 5.4395672	test: 5.6855277	best: 5.6853439 (297)	total: 15.2s	remaining: 2m 16s
400:	learn: 5.3607333	test: 5.6765532	best: 5.6757710 (388)	total: 20.2s	remaining: 2m 11s
500:	learn: 5.2814877	test: 5.6686129	best: 5.6681198 (493)	total: 25.2s	remaining: 2m 5s
600:	learn: 5.2017025	test: 5.6637599	best: 5.6637271 (599)	total: 30.2s	remaining: 2m
700:	learn: 5.1259405	test: 5.6592416	best: 5.6582643 (687)	total: 35.1s	remaining: 1m 55s
800:	learn: 5.0507910	test: 5.6624402	best: 5.6577275 (727)	total: 40.1s	remaining: 1m 49s
900:	learn: 4.9829272	test: 5.6605021	best: 5.6577275 (727)	total: 45.1s	remaining: 1m 45s
1000:	learn: 4.9092391	test: 5.6607638	best: 5.6577275 (727)	total: 50.1s	remaining: 1m 40s
110

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=8,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_x2.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.6661189	test: 16.7089618	best: 16.7089618 (0)	total: 52.5ms	remaining: 2m 37s
100:	learn: 5.6237135	test: 5.7337311	best: 5.7337311 (100)	total: 5.97s	remaining: 2m 51s
200:	learn: 5.4461440	test: 5.6970147	best: 5.6965375 (195)	total: 11.7s	remaining: 2m 43s
300:	learn: 5.3090347	test: 5.6787493	best: 5.6787406 (299)	total: 17.4s	remaining: 2m 36s
400:	learn: 5.1884374	test: 5.6716518	best: 5.6706650 (397)	total: 23.1s	remaining: 2m 29s
500:	learn: 5.0797222	test: 5.6672549	best: 5.6663698 (490)	total: 28.9s	remaining: 2m 23s
600:	learn: 4.9610948	test: 5.6641755	best: 5.6635718 (563)	total: 34.7s	remaining: 2m 18s
700:	learn: 4.8515345	test: 5.6657874	best: 5.6621934 (635)	total: 40.5s	remaining: 2m 12s
800:	learn: 4.7363956	test: 5.6627754	best: 5.6619279 (790)	total: 46.3s	remaining: 2m 7s
900:	learn: 4.6305780	test: 5.6651501	best: 5.6619279 (790)	total: 52s	remaining: 2m 1s
1000:	learn: 4.5331526	test: 5.6735724	best: 5.6619279 (790)	total: 57.8s	remaining: 1m 55s
11

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=6,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_x3.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.7222443	test: 16.7730550	best: 16.7730550 (0)	total: 39.3ms	remaining: 1m 57s
100:	learn: 5.7602225	test: 5.7729295	best: 5.7729295 (100)	total: 4.47s	remaining: 2m 8s
200:	learn: 5.6291979	test: 5.7059487	best: 5.7059487 (200)	total: 8.77s	remaining: 2m 2s
300:	learn: 5.5601242	test: 5.6888546	best: 5.6888546 (300)	total: 13.1s	remaining: 1m 57s
400:	learn: 5.4963716	test: 5.6735359	best: 5.6730622 (399)	total: 17.6s	remaining: 1m 53s
500:	learn: 5.4459872	test: 5.6662104	best: 5.6661759 (491)	total: 21.9s	remaining: 1m 49s
600:	learn: 5.3990209	test: 5.6633278	best: 5.6629650 (591)	total: 26.4s	remaining: 1m 45s
700:	learn: 5.3524321	test: 5.6613999	best: 5.6612832 (680)	total: 31s	remaining: 1m 41s
800:	learn: 5.3075007	test: 5.6550567	best: 5.6546848 (795)	total: 35.6s	remaining: 1m 37s
900:	learn: 5.2622164	test: 5.6538676	best: 5.6528149 (892)	total: 40.2s	remaining: 1m 33s
1000:	learn: 5.2226150	test: 5.6527591	best: 5.6523987 (933)	total: 44.8s	remaining: 1m 29s
11

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=5,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_x4.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.7851931	test: 16.8340837	best: 16.8340837 (0)	total: 37.5ms	remaining: 1m 52s
100:	learn: 5.8368201	test: 5.8307301	best: 5.8307301 (100)	total: 4.21s	remaining: 2m
200:	learn: 5.7040055	test: 5.7466747	best: 5.7466747 (200)	total: 8.16s	remaining: 1m 53s
300:	learn: 5.6428007	test: 5.7196086	best: 5.7193597 (293)	total: 12.1s	remaining: 1m 48s
400:	learn: 5.6008613	test: 5.7084989	best: 5.7083945 (399)	total: 16.1s	remaining: 1m 44s
500:	learn: 5.5627704	test: 5.6977980	best: 5.6976133 (494)	total: 19.9s	remaining: 1m 39s
600:	learn: 5.5263958	test: 5.6860419	best: 5.6857955 (598)	total: 23.8s	remaining: 1m 34s
700:	learn: 5.4978350	test: 5.6829370	best: 5.6829370 (700)	total: 27.9s	remaining: 1m 31s
800:	learn: 5.4683152	test: 5.6772921	best: 5.6772055 (793)	total: 31.9s	remaining: 1m 27s
900:	learn: 5.4422137	test: 5.6705155	best: 5.6705155 (900)	total: 35.9s	remaining: 1m 23s
1000:	learn: 5.4147992	test: 5.6674492	best: 5.6670056 (970)	total: 39.9s	remaining: 1m 19s
11

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:

# Initialize CatBoost regressor
model = CatBoostRegressor(
    iterations=3000,          # Number of boosting rounds
    learning_rate=0.1,        # Step size
    depth=3,                  # Depth of the trees
    cat_features=categorical_features,  # Categorical feature indices
    verbose=100,               # Prints training progress every 100 iterations
    task_type="GPU",
    devices='0'
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

# Make predictions
y_pred = model.predict(X_val)

# Evaluate model performance
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Save the trained model
model.save_model('catboost_model_x5.bin')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 16.9814983	test: 17.0312381	best: 17.0312381 (0)	total: 24.6ms	remaining: 1m 13s
100:	learn: 6.0786656	test: 6.0250764	best: 6.0250764 (100)	total: 2.42s	remaining: 1m 9s
200:	learn: 5.8521968	test: 5.8387561	best: 5.8387561 (200)	total: 4.68s	remaining: 1m 5s
300:	learn: 5.7821483	test: 5.7873140	best: 5.7873140 (300)	total: 7.04s	remaining: 1m 3s
400:	learn: 5.7437523	test: 5.7642619	best: 5.7641980 (398)	total: 9.42s	remaining: 1m 1s
500:	learn: 5.7159360	test: 5.7457370	best: 5.7457370 (500)	total: 11.7s	remaining: 58.5s
600:	learn: 5.6952444	test: 5.7338959	best: 5.7335059 (596)	total: 14s	remaining: 56s
700:	learn: 5.6768192	test: 5.7254808	best: 5.7254793 (699)	total: 16.3s	remaining: 53.5s
800:	learn: 5.6599237	test: 5.7174693	best: 5.7172956 (796)	total: 18.6s	remaining: 51.2s
900:	learn: 5.6450807	test: 5.7124261	best: 5.7123981 (899)	total: 20.9s	remaining: 48.8s
1000:	learn: 5.6351132	test: 5.7110105	best: 5.7109758 (999)	total: 23.2s	remaining: 46.4s
1100:	learn:

c:\Users\ferha\.conda\envs\datathon\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
